In [2]:
# src/evaluation/smci_evaluation.py

import numpy as np
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)


def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}


def classification_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}


def evaluate_strategy(y_true, y_pred, trading_days=252):
    """
    Rule: long when predicted next-day return > 0, otherwise flat.
    y_true / y_pred are daily log returns.
    """
    positions = (y_pred > 0).astype(int)
    strat_rets = positions * y_true

    sign_true = np.sign(y_true)
    sign_pred = np.sign(y_pred)
    mask = sign_true != 0
    hit_rate = np.mean(sign_true[mask] == sign_pred[mask]) if mask.sum() > 0 else np.nan

    mean_ret = strat_rets.mean()
    std_ret = strat_rets.std(ddof=1)
    sharpe = np.sqrt(trading_days) * mean_ret / std_ret if std_ret > 0 else np.nan

    cum_log = np.cumsum(strat_rets)
    wealth = np.exp(cum_log)
    running_max = np.maximum.accumulate(wealth)
    drawdowns = wealth / running_max - 1.0
    max_dd = drawdowns.min()

    return {
        "hit_rate": hit_rate,
        "mean_daily_ret": mean_ret,
        "annualized_sharpe": sharpe,
        "max_drawdown": max_dd,
    }


def equity_curve_from_predictions(y_true, y_pred):
    """Long if prediction > 0, else flat; return cumulative wealth series."""
    positions = (y_pred > 0).astype(int)
    strat_rets = positions * y_true
    cum_log = np.cumsum(strat_rets)
    wealth = np.exp(cum_log)
    return wealth
